In [ ]:
import pandas as pd
import numpy as np
import json
import toml
import math
import matplotlib.pyplot as plt
import subprocess
%matplotlib notebook

# Pathfinder Swerve Path Motion Profile Post Processing
This notebook is testing our post-processing of the trajectory segment array to modify the cruise speed for portions of a path in order to handle sharp corners better.

This uses our Java robot code and the [Pathfinder](https://github.com/JacisNonsense/Pathfinder) library to output trajectory points to a `.csv` file we use to plot.

The velocity and headings for each segment is transformed into a `forward` and `strafe` input to the swerve drive.

Parameters are:

- `dt` = iteration time, loop period (ms)
- `v_max` = max velocity (ft/sec)
- `a_max` = max acceleration (ft/sec/sec)
- `j_max` = max jerk (ft/sec/sec/sec)
- `waypoints` = x, y, and exit angle (degrees) 


In [ ]:
# drive forward down field
toml_str = """
waypoints = [
  {x = 0.0, y = 0.0, angle = 0.0},
  {x = 10.0, y = 0.0, angle = 0.0},

]
profile = [
  {start = 2.0, end = 3.0, velocity = 1.0 }
]
fit              = "HERMITE_CUBIC"
sample_count     = 20_000
dt               = 0.05  # sec
max_velocity     = 1.7   # m/sec
max_acceleration = 2.0   # m/sec/sec
max_jerk         = 60.0  # m/sec/sec/sec
"""

In [ ]:
# prereq: run 'gradle install' in top-level dir
process = subprocess.run(
    "MOTION_OPTS=-Djava.library.path={0}/lib {0}/bin/motion '{1}'".format(
        "../build/install/motion",
        json.dumps({"pathfinder": {"toml": toml_str}})),
    shell=True)

df = pd.read_csv("data/trajectory.csv")
path = toml.loads(toml_str)
wp = pd.DataFrame(path['waypoints'])

In [ ]:
df['heading_deg'] = df['heading'].apply(math.degrees)
df['velocity_normalized'] = df['velocity'] / df['velocity'].max()
df['forward'] = df['heading'].apply(math.cos) * df['velocity_normalized']
df['strafe'] = -df['heading'].apply(math.sin) * df['velocity_normalized']

df.loc[:,['velocity', 'acceleration']].plot()
plt.grid(True, color='lightgray', linestyle='--', linewidth=0.6)
plt.xlabel("Time (ms)")
l = plt.legend(["Velocity ($m/sec$)", "Acceleration ($m/sec^2$)"])

In [ ]:
df[['heading_deg', 'velocity', 'acceleration', 'velocity_normalized', 'forward', 'strafe']]